In [2]:
!git clone https://github.com/viniciusmgn/uaibot_vinicius.git
!pip install uaibot_vinicius/

Cloning into 'uaibot_vinicius'...


Processing c:\users\isadora\documents\engenharia\9° período\manipuladores\tp-manipuladores\uaibot_vinicius
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


  Created wheel for uaibot: filename=uaibot-0.0.1-py3-none-any.whl size=105657 sha256=3be28be5de7d72c3f15ac912c2f990f32753fa5f64c6ff0c8e3ac4263d04fbce
  Stored in directory: C:\Users\Isadora\AppData\Local\Temp\pip-ephem-wheel-cache-s1pdubkv\wheels\68\62\d7\374424dc50316ac86a3788a89d9431332c344d67ae55e1ac27
Successfully built uaibot
  Attempting uninstall: uaibot
    Found existing installation: uaibot 0.0.1
    Uninstalling uaibot-0.0.1:
      Successfully uninstalled uaibot-0.0.1


In [3]:
import uaibot as ub
import numpy as np


dt = 0.01
t = 0
tmax = 6
x = 0
y = 0.3
z = -0.3
R = 0.03

def get_configuration(robot):
  return robot.q

def set_configuration_speed(robot, qdot_des):
  q_next = robot.q + qdot_des*dt
  robot.add_ani_frame(time = t+dt, q = q_next)


#As inicializações (ex: parâmetros do controlador) virão aqui
robot = ub.Robot.create_kuka_kr5()
box_a = ub.Box(name="box_a", width=1.5, depth=0.3, height=0.3, htm=ub.Utils.trn([0, 0.8, 0.15]),color="red")
box_b = ub.Box(name="box_b", width=0.3, depth=0.3, height=0.3, htm=ub.Utils.trn([0, -0.8, 0.15]),color="red")


ball = ub.Ball(name="ball",radius=0.05,mass=1,color="blue", htm=ub.Utils.trn([0.2, 0.8, 0.35]))
ball2 = ub.Ball(name="ball2",radius=0.05,mass=1,color="blue", htm=ub.Utils.trn([0.2, 0.8, 0.35]))
ball3 = ub.Ball(name="ball3",radius=0.05,mass=1,color="blue", htm=ub.Utils.trn([-0.4, 0.8, 0.35]))
ball4 = ub.Ball(name="ball4",radius=0.05,mass=1,color="blue", htm=ub.Utils.trn([0.4, 0.8, 0.35]))
cilindro = ub.Cylinder(name="cilindro",height=0.1,radius=0.05,mass=1,color="black",htm=ub.Utils.trn([0, -0.8, 0.35]))


#Especifica a pose desejada para o efetuador

htm_d = ub.Utils.trn([x,y,z]) * robot.fkm() 
    
#Faz a extração dos elementos x_d, y_d, z_d e s_d
x_d = htm_d[0:3,0]
y_d = htm_d[0:3,1]
z_d = htm_d[0:3,2]
s_d = lambda tt: np.matrix([R * tt, -0.8 + tt, 0.15 + tt]).reshape((3, 1))


#Cria um referencial no cenário para que seja possível ver se ele alcancou a
#pose
frame_d = ub.Frame(htm = htm_d)

#Cria a simulação
sim = ub.Simulation([robot,frame_d,box_a,ball,ball2,ball3,ball4,cilindro,box_b])

#Captura o número de juntas do robô
n = np.shape(robot.q)[0]

#Cria a função F:
def fun_F(r):
    A = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
    w_tol = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
    F = np.matrix(np.zeros((6, 1)))
    for i in range(6):
        if abs(r[i, 0]) < w_tol[i]:
            F[i, 0] = -A[i] * (r[i, 0] / w_tol[i])
        elif r[i, 0] >= w_tol[i]:
            F[i, 0] = -A[i]
        else:
            F[i, 0] = A[i]
    return F

#Cria uma matriz para o histórico de função de tarefa, da ação de controle
#e do tempo
hist_r = np.matrix(np.zeros((6,0)))
hist_u = np.matrix(np.zeros((n,0)))
hist_t = []

#Colocaremos aqui nosso "main" do controlador, que ficará em um laço
#durante um tempo tmax

for i in range(round(tmax/dt)):
    #################################
    # Início da lógica de controle  #
    #################################

    #Mede a configuração dos sensores
    q = get_configuration(robot)

    #Calcula a cinemática direta e Jacobiana para o efetuador nessa
    #configuração
    Jg, fk = robot.jac_geo(q)
    #Faz a extração de x_e, y_e, z_e e s_e
    x_e = fk[0:3,0]
    y_e = fk[0:3,1]
    z_e = fk[0:3,2]
    s_e = fk[0:3,3]

    #Monta o vetor de tarefa
    r = np.matrix(np.zeros((6,1)))
    
    r[0:3] = s_e - s_d(t)
    r[3] = 1- x_d.T * x_e
    r[4] = 1- y_d.T * y_e
    r[5] = 1- z_d.T * z_e

    #Monta a Jacobiana de tarefa
    Jr = np.matrix(np.zeros((6,n)))
    Jr[0:3,:] = Jg[0:3,:]
    Jr[3,:] = x_d.T * ub.Utils.S(x_e) * Jg[3:6,:]
    Jr[4,:] = y_d.T * ub.Utils.S(y_e) * Jg[3:6,:]
    Jr[5,:] = z_d.T * ub.Utils.S(z_e) * Jg[3:6,:]

    #Calcula a ação de controle
    u = ub.Utils.dp_inv(Jr,0.001)*fun_F(r)

    #Guarda informações no histórico
    hist_r = np.block([hist_r, r])
    hist_u = np.block([hist_u, u])
    hist_t.append(t)

    #Manda a ação de controle para o robô
    set_configuration_speed(robot, u)

    #################################
    # Fim da lógica de controle     #
    #################################
    ball.add_ani_frame(t, htm = ub.Utils.trn((s_e - s_d(t))/2))
    #O tempo sempre vai passar no final do ciclo
    t+=dt
    

#Roda a simulação
sim.run()

#plota os gráficos
ub.Utils.plot(hist_t, hist_r, "", "Tempo (s)", "Função de tarefa", "r")
ub.Utils.plot(hist_t, hist_u, "", "Tempo (s)", "Ação (rad/s ou m/s)", "u")